# Business Problem - how venues influence the price of real estate? 

For this project, I decided to model the relation between the price of an estate and the type of venues in its neighborhood. 
The objective is to determine whether or not some venues have a direct impact on the price, and if so how is the price affected by it. 

Concretely I will have to: 
1. geolocalize a set of real-estate transactions. As I cannot use geocoder API at such a large scale (too much data), I will have to make the analysis at the level of the neighborhoods. 
2. Calculate the average price per square meter for all neighborhoods in my data set. 
3. List the venues (and the type of venues) for each neighborhood
4. Concatenate my two sources of data into one df
5. Split my data in order to have a train set and a test set 
6. Pick and design the right algorithm to determine & predict the price of an estate given the venues in its neighborhood
7. Evaluate the accuracy of the model

Note: 
1. To make this analysis accurate and a bit pertinent I would need to work at the level of each transaction (i.e. geolocalize each real estate), because not only is it the closest neighborhood that has an impact, but also the distance is a fluctuant & quite relevant parameter (e.g. it may be convinent to have a supermarket nearby your place but maybe not to having seen on the building from your living room. You'd rather have a beautiful park). 
2. Some other factors will be disregarded to make this study simplier. Yet they probably have some big impacts on the price (e.g. construction date of the building, material used for the construction, ecominical & social indicators of the population living in the neighborhood, etc.)

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans



# Presentation of my Data & its pre-processing

I will first import my main data source as a csv using the following method. Please find below a data set description: 

Manhattan Rolling Sales File.  All Sales From Oct 2018 - Sep 2019.			
"For sales prior to the Final, Neighborhood Name and Descriptive Data reflect the Final Roll 2019/20.  
Sales after the Final Roll, Neighborhood Name and Descriptive Data reflect current data"			
Building Class Category is based on Building Class at Time of Sale.			
Note: Condominium and cooperative sales are on the unit level and understood to have a count of one.			


If you are interested by this dataset, you can download it from this url: https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page

In [31]:
# The code was removed by Watson Studio for sharing.

,NEIGHBORHOOD,ADDRESS,ZIP CODE,GROSS_SQUARE_FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE_PRICE
0,ALPHABET CITY,743 EAST 6TH STREET,10009,3.68,1940,1,S1,"3,200,000"
1,ALPHABET CITY,526 EAST 5TH STREET,10009,5.2,1900,1,A4,"6,100,000"
2,ALPHABET CITY,263 EAST 7TH STREET,10009,3.6,1899,1,C0,"6,300,000"
3,ALPHABET CITY,300 EAST 3RD STREET,10009,7.989,2001,2,C1,"1,950,000"
4,ALPHABET CITY,332 EAST 4TH STREET,10009,17.478,1920,2,C7,"14,000,000"


Let’s have a look at data dimensionality, feature names, and feature types

In [32]:
df_data.shape

(6835, 8)

From the output, we can see that the table contains 6835 rows and 8 columns.

We can use the info() method to output some general information about the dataframe:

In [33]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6835 entries, 0 to 6834
Data columns (total 8 columns):
NEIGHBORHOOD                      6835 non-null object
ADDRESS                           6835 non-null object
ZIP CODE                          6835 non-null int64
GROSS_SQUARE_FEET                 6835 non-null object
YEAR BUILT                        6835 non-null int64
TAX CLASS AT TIME OF SALE         6835 non-null int64
BUILDING CLASS AT TIME OF SALE    6835 non-null object
SALE_PRICE                        6835 non-null object
dtypes: int64(3), object(5)
memory usage: 427.3+ KB


I will clean a bit my df by removing some useless columns (as I explained in my presentation, I will model a price function only based on a few variables)

In [3]:
df_data_0 = df_data_0.drop(["ADDRESS","ZIP CODE","YEAR BUILT","TAX CLASS AT TIME OF SALE", "BUILDING CLASS AT TIME OF SALE"], axis=1)
df_data_0.head()

,NEIGHBORHOOD,GROSS_SQUARE_FEET,SALE_PRICE
0,ALPHABET CITY,3.68,"3,200,000"
1,ALPHABET CITY,5.2,"6,100,000"
2,ALPHABET CITY,3.6,"6,300,000"
3,ALPHABET CITY,7.989,"1,950,000"
4,ALPHABET CITY,17.478,"14,000,000"


In order to compute the quantities, I first need to transform the type (once into str to remove the comma; then a second time into float to apply some basic statistical functions)

In [4]:
df_data_0['SALE_PRICE'] = df_data_0.SALE_PRICE.astype(str)
df_data_0['GROSS_SQUARE_FEET'] = df_data_0.GROSS_SQUARE_FEET.astype(str)

a = df_data_0['SALE_PRICE'].tolist() 
b = df_data_0['GROSS_SQUARE_FEET'].tolist()

a = [i.replace(",","") for i in a]
b = [i.replace(",","") for i in b]

df_data_0['SALE_PRICE'] = a
df_data_0['GROSS_SQUARE_FEET'] = b

df_data_0['SALE_PRICE'] = df_data_0.SALE_PRICE.astype(float)
df_data_0['GROSS_SQUARE_FEET'] = df_data_0.GROSS_SQUARE_FEET.astype(float)

df_data_0.head()

,NEIGHBORHOOD,GROSS_SQUARE_FEET,SALE_PRICE
0,ALPHABET CITY,3.680,3200000.0
1,ALPHABET CITY,5.200,6100000.0
2,ALPHABET CITY,3.600,6300000.0
3,ALPHABET CITY,7.989,1950000.0
4,ALPHABET CITY,17.478,14000000.0


As I'm facing some performance issues with geocoder API, I will use a ready-made data set where I can extract the geographic coordinates for each neighborhood in Manhattan

In [5]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [6]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [7]:
neighborhoods_data = newyork_data['features']

In [8]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [9]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [10]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [11]:
manhattan_data.rename(columns={'Neighborhood': 'NEIGHBORHOOD'},inplace=True)
manhattan_data = manhattan_data.drop("Borough", axis=1)
manhattan_data['NEIGHBORHOOD'] = manhattan_data['NEIGHBORHOOD'].str.upper() 
manhattan_data.head()

,NEIGHBORHOOD,Latitude,Longitude
0,MARBLE HILL,40.876551,-73.910660
1,CHINATOWN,40.715618,-73.994279
2,WASHINGTON HEIGHTS,40.851903,-73.936900
3,INWOOD,40.867684,-73.921210
4,HAMILTON HEIGHTS,40.823604,-73.949688


In [12]:
Lastdf = df_data_0.merge(manhattan_data)
Lastdf.head()

,NEIGHBORHOOD,GROSS_SQUARE_FEET,SALE_PRICE,Latitude,Longitude
0,CIVIC CENTER,3.764,7079588.0,40.715229,-74.005415
1,CIVIC CENTER,3.764,7079588.0,40.715229,-74.005415
2,CIVIC CENTER,3.827,7900000.0,40.715229,-74.005415
3,CIVIC CENTER,3.827,7900000.0,40.715229,-74.005415
4,CIVIC CENTER,2.220,4700000.0,40.715229,-74.005415


In [13]:
Lastdf['Price/m2'] = (Lastdf.SALE_PRICE / Lastdf.GROSS_SQUARE_FEET)

In [14]:
g = Lastdf.groupby('NEIGHBORHOOD')

In [15]:
g.mean()

,GROSS_SQUARE_FEET,SALE_PRICE,Latitude,Longitude,Price/m2
NEIGHBORHOOD,,,,,
CIVIC CENTER,4.423667,7.026654e+06,40.715229,-74.005415,2.056424e+06
CLINTON,437.652183,2.426260e+06,40.759101,-73.996119,5.123421e+05
EAST VILLAGE,225.606312,5.605654e+06,40.727847,-73.982226,1.017368e+06
FLATIRON,168.858676,1.542258e+07,40.739673,-73.990947,2.340082e+06
GRAMERCY,351.718578,4.411438e+06,40.737210,-73.981376,9.684232e+05
INWOOD,103.560471,2.679084e+06,40.867684,-73.921210,1.268207e+05
LITTLE ITALY,280.019705,5.945024e+06,40.719324,-73.997305,1.501599e+06
LOWER EAST SIDE,416.199264,3.293915e+06,40.717807,-73.980890,7.296687e+05
MANHATTAN VALLEY,472.416203,1.585142e+06,40.797307,-73.964286,1.921495e+05


# Extract Data from FourSquare & Merge them with my Df

Now that I have preprocessed my data, I need to get the venues for each neighborhood

In [16]:
CLIENT_ID = 'U1JPB1RU3PAUAYPENRWH3PAQB0KA4CKW2KZSXXTKDKTM5M3Y' # your Foursquare ID
CLIENT_SECRET = 'C05RBLVLNZ4V0AMBQPOSPSOV10HUJIOQI5FCUYTG3CQZORNR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: U1JPB1RU3PAUAYPENRWH3PAQB0KA4CKW2KZSXXTKDKTM5M3Y
CLIENT_SECRET:C05RBLVLNZ4V0AMBQPOSPSOV10HUJIOQI5FCUYTG3CQZORNR


In [17]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
Limit = 5 
radius = 500 

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            Limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [19]:
Sample = Lastdf.sample(n = 200)

In [20]:
manhattan_venues = getNearbyVenues(names=Sample['NEIGHBORHOOD'],
                                   latitudes = Sample['Latitude'],
                                   longitudes= Sample['Longitude']
                                  )


SOHO
EAST VILLAGE
TRIBECA
MURRAY HILL
SOHO
SOHO
LOWER EAST SIDE
LOWER EAST SIDE
MANHATTAN VALLEY
ROOSEVELT ISLAND
MURRAY HILL
CLINTON
SOHO
TRIBECA
LOWER EAST SIDE
SOHO
MURRAY HILL
SOHO
TRIBECA
MURRAY HILL
SOHO
TRIBECA
GRAMERCY
TRIBECA
SOHO
SOHO
GRAMERCY
TRIBECA
MURRAY HILL
TRIBECA
MANHATTAN VALLEY
CLINTON
EAST VILLAGE
TRIBECA
TRIBECA
EAST VILLAGE
TRIBECA
MURRAY HILL
TRIBECA
MURRAY HILL
EAST VILLAGE
LITTLE ITALY
CLINTON
GRAMERCY
GRAMERCY
TRIBECA
CLINTON
MANHATTAN VALLEY
LOWER EAST SIDE
LOWER EAST SIDE
GRAMERCY
TRIBECA
TRIBECA
TRIBECA
LOWER EAST SIDE
EAST VILLAGE
TRIBECA
CIVIC CENTER
LOWER EAST SIDE
FLATIRON
FLATIRON
LOWER EAST SIDE
LOWER EAST SIDE
LOWER EAST SIDE
MANHATTAN VALLEY
FLATIRON
GRAMERCY
FLATIRON
MURRAY HILL
LOWER EAST SIDE
GRAMERCY
TRIBECA
LOWER EAST SIDE
LOWER EAST SIDE
TRIBECA
FLATIRON
GRAMERCY
MURRAY HILL
LOWER EAST SIDE
SOHO
LOWER EAST SIDE
MURRAY HILL
GRAMERCY
GRAMERCY
CLINTON
FLATIRON
LOWER EAST SIDE
LOWER EAST SIDE
TRIBECA
TRIBECA
SOHO
LOWER EAST SIDE
TRIBECA
MURRAY HI

In [23]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Art Gallery,Arts & Crafts Store,Bar,Beer Bar,Beer Store,Coffee Shop,Comedy Club,Cycle Studio,Dance Studio,Dessert Shop,Dog Run,Falafel Restaurant,Farmers Market,Filipino Restaurant,Fried Chicken Joint,Furniture / Home Store,Gourmet Shop,Greek Restaurant,Gym,Gym / Fitness Center,Hawaiian Restaurant,Hostel,Hotel,Italian Restaurant,Japanese Restaurant,Mediterranean Restaurant,Miscellaneous Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Outdoors & Recreation,Park,Peruvian Restaurant,Pet Café,Playground,Poke Place,Salad Place,Salon / Barbershop,Sandwich Place,Spa,Spanish Restaurant,Tea Room,Theater,Vietnamese Restaurant,Waterfront,Wine Bar,Yoga Studio
0,SOHO,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,SOHO,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,SOHO,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,SOHO,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,SOHO,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [29]:
manhattan_onehot.rename(columns={'Neighborhood': 'NEIGHBORHOOD'},inplace=True)
 
h = Sample.merge(manhattan_onehot)

h.head()

,NEIGHBORHOOD,GROSS_SQUARE_FEET,SALE_PRICE,Latitude,Longitude,Price/m2,Art Gallery,Arts & Crafts Store,Bar,Beer Bar,Beer Store,Coffee Shop,Comedy Club,Cycle Studio,Dance Studio,Dessert Shop,Dog Run,Falafel Restaurant,Farmers Market,Filipino Restaurant,Fried Chicken Joint,Furniture / Home Store,Gourmet Shop,Greek Restaurant,Gym,Gym / Fitness Center,Hawaiian Restaurant,Hostel,Hotel,Italian Restaurant,Japanese Restaurant,Mediterranean Restaurant,Miscellaneous Shop,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Outdoors & Recreation,Park,Peruvian Restaurant,Pet Café,Playground,Poke Place,Salad Place,Salon / Barbershop,Sandwich Place,Spa,Spanish Restaurant,Tea Room,Theater,Vietnamese Restaurant,Waterfront,Wine Bar,Yoga Studio
0,SOHO,258.0,2650000.0,40.722184,-74.000657,10271.317829,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,SOHO,258.0,2650000.0,40.722184,-74.000657,10271.317829,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,SOHO,258.0,2650000.0,40.722184,-74.000657,10271.317829,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,SOHO,258.0,2650000.0,40.722184,-74.000657,10271.317829,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,SOHO,258.0,2650000.0,40.722184,-74.000657,10271.317829,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
